In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

In [3]:
import numpy
import pandas

print("✅ numpy version:", numpy.__version__)
print("✅ pandas version:", pandas.__version__)


✅ numpy version: 1.24.4
✅ pandas version: 1.5.3


In [4]:
pip install pandas openpyxl pillow requests


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%%time
import pandas as pd
from openpyxl import load_workbook
from PIL import Image
import io, base64, requests, os, logging

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(message)s',
    datefmt='%H:%M:%S'
)

def extract_text_and_table(file_path):
    xls = pd.ExcelFile(file_path)
    results = []

    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet, dtype=str)
        flat_text = sorted(set(val.strip() for val in df.astype(str).stack() if val and val.strip()))
        table_markdown = df.to_markdown(index=False)

        summary_lines = []
        table_type = ""
        description_lines = []

        numeric_df = df.apply(pd.to_numeric, errors='coerce')
        numeric_cols = [col for col in numeric_df.columns if numeric_df[col].notna().sum() > 0]
        shape_desc = f"-  테이블 형태: {df.shape[0]}행 × {df.shape[1]}열"
        summary_lines.append(shape_desc)

        # 테이블 유형
        if "Department" in df.columns:
            table_type = "부서별 비교 테이블 (범주형 행 vs 수치형 열)"
        elif df.dtypes[0] == 'object':
            table_type = "범주형 키에 수치 값을 가진 테이블"
        else:
            table_type = "수치 중심의 행렬 또는 측정 테이블"

        summary_lines.append(f"-  테이블 유형: {table_type}")

        if numeric_cols:
            for col in numeric_cols:
                max_val = numeric_df[col].max()
                avg_val = numeric_df[col].mean()
                summary_lines.append(f"-  열 '{col}': 최대값={max_val}, 평균={avg_val:.2f}")

        # 행별 요약
        if len(numeric_cols) >= 1:
            key_col = df.columns[0]
            for idx, row in df.iterrows():
                parts = [f"{row[key_col]}:"]
                for col in numeric_cols:
                    val = row[col]
                    if pd.notna(val):
                        parts.append(f"{col} {val}")
                description_lines.append(" / ".join(parts))

        results.append({
            "sheet": sheet,
            "text": flat_text,
            "text_summary": f"- 이 시트에는 총 {len(flat_text)}개의 고유 텍스트 항목이 있습니다.",
            "table_type": table_type,
            "table_summary": "\n".join(summary_lines),
            "table_description": "\n".join(description_lines),
            "table_markdown": table_markdown
        })

    return results

def extract_images_from_excel(file_path):
    wb = load_workbook(file_path)
    imgs = []
    for sheet in wb.worksheets:
        for img in getattr(sheet, "_images", []):
            if hasattr(img, "_data"):
                img_bytes = img._data()
                pil = Image.open(io.BytesIO(img_bytes)).convert("RGB")
                pil.thumbnail((640, 480))
                imgs.append({"sheet": sheet.title, "image": pil})
    return imgs

def run_qwen_vl_on_chart_image(pil_image):
    buf = io.BytesIO()
    pil_image.save(buf, format="PNG")
    img_b64 = base64.b64encode(buf.getvalue()).decode("utf-8")

    prompt = (
        "이 이미지는 엑셀 문서에 포함된 차트 또는 표 이미지입니다.\n"
        "다음 항목을 한국어로 정확하게 설명해주세요:\n"
        "1. 차트나 표의 제목 또는 주제는 무엇인가요?\n"
        "2. X축과 Y축은 각각 무엇을 의미하나요?\n"
        "3. X축 항목별로 상응하는 Y값(숫자)을 구체적으로 나열해주세요.\n"
        "4. 눈에 띄는 수치나 패턴은 무엇인가요?\n"
        "5. 전체 내용을 한두 문장으로 요약해주세요."
    )

    res = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "qwen2.5vl",
            "prompt": prompt,
            "images": [img_b64],
            "stream": False
        }
    )
    if res.status_code == 200:
        return res.json().get("response", "").strip()
    else:
        raise RuntimeError(f"Ollama 호출 실패: {res.status_code}")

def assemble_excel_report(file_path):
    text_table = extract_text_and_table(file_path)
    images = extract_images_from_excel(file_path)

    report = ""

    for sheet_info in text_table:
        name = sheet_info["sheet"]
        report += f"\n\n## 📄 시트: {name}\n"
        report += "\n### 📝 텍스트 항목\n"
        report += "\n".join(f"- {t}" for t in sheet_info["text"])
        report += "\n\n###  텍스트 요약\n" + sheet_info["text_summary"]
        report += "\n\n###  테이블 유형\n" + sheet_info["table_type"]
        report += "\n\n###  테이블 요약\n" + sheet_info["table_summary"]
        report += "\n\n###  행별 설명\n" + sheet_info["table_description"]
        report += "\n\n###  마크다운 테이블\n" + sheet_info["table_markdown"]

    for i, img in enumerate(images):
        logging.info(f" 이미지 {i+1}/{len(images)} 해석 중...")
        try:
            analysis = run_qwen_vl_on_chart_image(img["image"])
            report += f"\n\n##  차트/표 이미지 {i+1} (시트: {img['sheet']})\n{analysis}"
        except Exception as e:
            report += f"\n\n##  이미지 해석 실패 {i+1}: {e}"

    return report.strip()

if __name__ == "__main__":
    file_path = r"C:\Ai_x\source\프로젝트2\sample_report_english.xlsx"
    if os.path.exists(file_path):
        logging.info(f" 분석 시작: {file_path}")
        print(assemble_excel_report(file_path))
    else:
        logging.error(" 파일을 찾을 수 없습니다.")


[10:10:45] 📂 분석 시작: C:\Ai_x\source\프로젝트2\sample_report_english.xlsx
[10:10:46] 📷 이미지 1/1 해석 중...


## 📄 시트: Report

### 📝 텍스트 항목
- -5%
- 0%
- 10%
- 11
- 12
- 150
- 220
- 25%
- 310
- 350
- 4
- 5
- 5%
- 9
- 90
- Department
- Development
- Employees
- Finance
- HR
- Marketing
- Sales
- Sales (Million $)
- The development team showed significant growth, while the marketing team experienced a slight decline compared to the previous year.
- This report summarizes the sales performance and number of employees in each department from January to June 2025.
- YoY Change
- nan

### 🔎 텍스트 요약
- 🧾 이 시트에는 총 27개의 고유 텍스트 항목이 있습니다.

### 📊 테이블 유형
범주형 키에 수치 값을 가진 테이블

### 📈 테이블 요약
- 📐 테이블 형태: 9행 × 4열
- 📌 테이블 유형: 범주형 키에 수치 값을 가진 테이블
- 🔍 열 'Unnamed: 1': 최대값=12.0, 평균=8.20
- 🔍 열 'Unnamed: 2': 최대값=350.0, 평균=224.00

### 📘 행별 설명
This report summarizes the sales performance and number of employees in each department from January to June 2025.:
The development team showed significant growth, while the marketing team experienced a slight decline compared to the previous year.:
nan:
Department: / Unnamed: 1 Emplo